# Exercise 02 - Paths, centralities, and community structure

In this week's assignment we explore some of the key network analytic concepts introduced in lecture 02 in practice. We will calculate path-based network characteristics for empirical data sets and we develop a simple approach to detect community structures based on the heuristic optimisation of modularity. We finally calculate and visualise the degree distribution of real networks. We will use the following five empirical data sets:

1) the collaboration network of the OpenSource software community `kde`  
2) the collaboration network of the OpenSource software community `gentoo`  
3) the power grid of the western states of the USA  
4) the contact network of students in a highschool  
5) an information sharing network of physicians in the United States  

All of these data are available in a single SQLite database file, which you can find in Moodle.

## Task 1: Paths, diameter, and components

### 1.1 Connected components

Implement Tarjan's algorithm for the calculation of connected components for an instance of `pathpy.Network` (see e.g. [Hopcroft and Tarjan 1973](https://dl.acm.org/citation.cfm?doid=362248.362272). Test your algorithm in the small toy example from lecture 2 (slide 14). Define a function that returns the relative size of the largest connected component (lcc) for a given network.

Compare your results with the implementation given in pathpy, using the function `reduce_to_gcc` in `pathpy.algorithms.components`.

In [1]:
import numpy as np
import pathpy as pp

import sys
sys.setrecursionlimit(1500)

def lcc_size(n, verbose = False):
    # Tarjan's algorithm traverses a network with a depth-first-search.
    # Whenever we discover a new node, we assign a "dfs number" that is incremented. 
    # We recursively call the tarjan_visit function on that node. Each node is further assigned a 
    # "low link" number, which stores the minimum dfs number that can be reached from that node 
    # using at most one back edge, see https://en.wikipedia.org/wiki/Depth-first_search#Output_of_a_depth-first_search
    # We update this number whenever we find a link to an already discovered node.

    # The algorithm considers each node and link exactly once, so for a network with n nodes and m 
    # links its complexity is O(n+m).
    
    # this number is incremented whenever a new node is discovered
    i = 1

    # we store the current value of i as "dfs number" once a node is first discovered
    dfs_num = {}

    # low link as explained above
    low_link = {}

    # when the call to tarjan_visit finishes, the stack contains all nodes in the current connected component
    # conveniently, python arrays can be used as stacks by calling the append() and pop() functions
    stack = []
    
    components = {}
    component_sizes = {}

    def tarjan_visit(network, v):
        """Recursive method of Tarjan's algorithm that generates all nodes 
        that are in the same (strongly) connected component as node v."""
        nonlocal dfs_num
        nonlocal low_link
        nonlocal stack
        nonlocal i
        nonlocal components
        nonlocal component_sizes

        # start with node v
        dfs_num[v] = i
        low_link[v] = i
        stack.append(v)
        i += 1

        # for all successors w of v, recursively call function
        # if w has not been previously discovered
        for w in network.successors[v]:

            # if w has not been previously discovered
            # recursively apply tarjan_visit to w
            if w not in dfs_num:
                tarjan_visit(network, w)
                
                # update low_link[v] since we can reach w from v
                low_link[v] = min(low_link[v], low_link[w])

            # we discovered a link to an already discovered node w
            elif w in stack:
                
                # update low_link[v] since we can reach w from v
                low_link[v] = min(low_link[v], dfs_num[w])
        
        # a "root" node completed a DFS traversal
        if low_link[v] == dfs_num[v]:
            # stack contains nodes that are in same 
            # strongly connected component as v
            components[v] = set()
            while True:
                x = stack.pop()
                components[v].add(x)
                if x == v:
                    break
            component_sizes[v] = len(components[v])
    
    for v in n.nodes:
        # visit node if it has not been visited yet
        if v not in dfs_num:
            tarjan_visit(n, v)
    
    if verbose:        
        print(components)
        for v in n.nodes:
            print('{0} -> ({1}, {2})'.format(v, dfs_num[v], low_link[v]))
    
    return max(component_sizes.values())/n.ncount()

In [ ]:
stop

In [2]:
# Example from L02 slide 22 with two connected components
n = pp.Network()
n.add_edge('a', 'b')
n.add_edge('b', 'c')
n.add_edge('a', 'c')
n.add_edge('d', 'f')
n.add_edge('d', 'g')
n.add_edge('d', 'e')
n.add_edge('e', 'f')
n.add_edge('f', 'g')

print(lcc_size(n))
n

0.5714285714285714


In [3]:
n

In [4]:
pp.algorithms.components.connected_components??

In [5]:
paths = pp.Paths()
paths.add_path('a,b,c,a')
paths.add_path('d,e,f,g,d,f')
pp.Network().from_paths(paths).to_undirected()

In [6]:
# Directed network example
n = pp.Network(directed=True)
n.add_edge('a', 'b')
n.add_edge('b', 'a')
n.add_edge('a', 'c')
n.add_edge('b', 'c')
n.add_edge('c', 'd')
n.add_edge('d', 'c')
n

In [7]:
print(lcc_size(n, verbose=True))

{'c': {'d', 'c'}, 'a': {'b', 'a'}}
a -> (1, 1)
b -> (4, 1)
c -> (2, 2)
d -> (3, 2)
0.5


In [8]:
n = pp.Network(directed=True)
n.add_edge('a', 'b')
n.add_edge('a', 'c')
n.add_edge('b', 'd')
n.add_edge('b', 'e')
n.add_edge('d', 'b')
n.add_edge('d', 'e')
n.add_edge('e', 'd')
n

In [9]:
print(lcc_size(n, verbose=True))

{'c': {'c'}, 'b': {'b', 'd', 'e'}, 'a': {'a'}}
a -> (1, 1)
b -> (3, 3)
c -> (2, 2)
d -> (4, 3)
e -> (5, 4)
0.6


### 1.2 Connected components in empirical data

Read the five data sets from the SQLite database as *undirected* networks and compute the sizes of connected components. Would you say that these networks contain a *giant* connected component?

In [10]:
import sqlite3

con = sqlite3.connect('data/01_networks.db')
con.row_factory = sqlite3.Row

n_highschool = pp.Network.from_sqlite(con.execute('SELECT source, target FROM highschool'), directed=False)
n_kde = pp.Network.from_sqlite(con.execute('SELECT source, target FROM kde'), directed=False)
n_gentoo = pp.Network.from_sqlite(con.execute('SELECT source, target FROM gentoo'), directed=False)
n_powergrid = pp.Network.from_sqlite(con.execute('SELECT source, target FROM powergrid'), directed=False)
n_physicians = pp.Network.from_sqlite(con.execute('SELECT source, target FROM physicians'), directed=False)

OperationalError: unable to open database file

In [ ]:
pp.algorithms.components.reduce_to_gcc

In [13]:
n_highschool

In [14]:
print(lcc_size(n_highschool))

1.0


In [15]:
print(lcc_size(n_kde))

1.0


In [16]:
print(lcc_size(n_kde))

1.0


In [17]:
print(lcc_size(n_powergrid))

1.0


In [18]:
print(lcc_size(n_physicians))

0.4854771784232365


### Task 1.3 Diameter and average path length in empirical data

Use the functions `diameter` and `avg_path_length` in `shortest_paths` in `pathpy.algorithms.shortest_paths` to compute the diameter and the average shortest path length of the (largest connected component in the) `physicians`, `highschool`, and `gentoo` data. Interpret the differences while considering the different sizes of the network.

In [19]:
print("Diameter of highschool is ", pp.algorithms.shortest_paths.diameter(n_highschool))
print("Diameter of physicians is ", pp.algorithms.shortest_paths.diameter(n_physicians))
print("Diameter of gentoo is ", pp.algorithms.shortest_paths.diameter(n_gentoo))

print("Avg. path length of highschool is ", pp.algorithms.shortest_paths.avg_path_length(n_highschool))
print("Avg. path length of physicians is ", pp.algorithms.shortest_paths.avg_path_length(n_physicians))
print("Avg. path length of gentoo is ", pp.algorithms.shortest_paths.avg_path_length(n_gentoo))

Diameter of highschool is  12.0
Diameter of physicians is  inf
Diameter of gentoo is  8.0
Avg. path length of highschool is  5.3180555555555555
Avg. path length of physicians is  inf
Avg. path length of gentoo is  3.101921691531873


In [ ]:
n_physicians

## Task 2: Modularity-based community detection

### 2.1 Partition Quality

Implement the partition quality function $Q(n, C)$ for a given network `n` and a non-overlapping partitioning `C` of nodes into communities as introduced in lecture 2 on slide 22. Test your method in the toy example with the partitioning depicted on slide 26. Check whether you obtain the same value for the partition quality.

In [20]:
def Q(network, C):
    m = network.ecount()
    n = network.ncount()    
    A = network.adjacency_matrix(weighted=False)
    idx = network.node_to_name_map()
    nodes = [v for v in idx]
    q = 0.0
    for v in network.nodes:
        for w in network.nodes:
            if C[v] == C[w]:
                q += A[idx[v], idx[w]] - network.nodes[v]['degree']*network.nodes[w]['degree']/(2*m)
    q /= 2*m
    return q

In [21]:
n

In [23]:
n

In [22]:
n = pp.Network()
n.add_edge('a', 'b')
n.add_edge('b', 'c')
n.add_edge('a', 'c')
n.add_edge('b', 'd')
n.add_edge('d', 'f')
n.add_edge('d', 'g')
n.add_edge('d', 'e')
n.add_edge('e', 'f')
n.add_edge('f', 'g')

communities = {}
for x in ['a', 'b', 'g']:
    communities[x] = 1
for x in ['d', 'e', 'f', 'c']:
    communities[x] = 2

q = Q(n, communities)
print(q)

-0.08024691358024688


### 2.2 Modularity Optimisation

Implement a simple heuristic optimisation algorithm to calculate the optimal modularity $Q_{opt}$ across all partitions. The idea of this algorithm is as follows: 

1) Start with a partitioning where you place each node in a separate community   
2) Draw two communities uniformly at random and merge them to a single community iff this merge increases partition quality   
3) Repeat the second step for a given number of iterations and output the final partitioning and partition quality  

Use your method to calculate $Q_{opt}$ for the toy example and plot the detected communities by coloring the nodes appropriately.

In [24]:
def Q_merge(network, C, merge=set()):
    m = network.ecount()
    n = network.ncount()    
    A = network.adjacency_matrix(weighted=False)
    idx = network.node_to_name_map()
    q = 0.0
    for v in network.nodes:
        for w in network.nodes:
            if C[v] == C[w] or (C[v] in merge and C[w] in merge):
                q += A[idx[v], idx[w]] - network.nodes[v]['degree']*network.nodes[w]['degree']/(2*m)
    q /= 2*m
    return q

def find_communities(network, iterations=100):
    
    # start with each node being in a separate cluster
    C = {}
    community_to_nodes = {}
    c = 0
    for n in network.nodes:
        C[n] = c
        community_to_nodes[c] = set([n])
        c += 1
    q = Q(network, C)
    communities = list(C.values())
    
    for i in range(iterations):
        # randomly choose two communities
        x, y = np.random.choice(communities, size=2)
        # check Q of merged communities
        q_new = Q_merge(network, C, merge=set([x, y]))
        if q_new > q:
            # actually merge the communities
            for n in community_to_nodes[x]:
                C[n] = y
            community_to_nodes[y] = community_to_nodes[y] | community_to_nodes[x]
            q = q_new
            communities.remove(x)
            del community_to_nodes[x]
    return C, q

To make this task a bit simpler, I provide you with the following method, which generates a community-based node-color mapping that you can use to color nodes according to (a maximum of 20) communities (hint: use the `node_color` parameter of the `pathpy.visualisation.plot` function).

In [25]:
def map_colors(n, communities):
    colors = ['red', 'green', 'blue', 'orange', 'yellow', 'cyan', 'blueviolet', \
              'chocolate', 'magenta', 'navy', 'plum', 'thistle', 'wheat', 'turquoise', \
              'steelblue', 'grey', 'powderblue', 'orchid', 'mintcream', 'maroon']
    node_colors = {}
    community_color_map = {}
    i = 0
    for v in n.nodes:
        if communities[v] not in community_color_map:
            community_color_map[communities[v]] = i%len(colors)
            i += 1
        node_colors[v] = colors[community_color_map[communities[v]]]
    return node_colors

In [26]:
n

In [36]:
C, q_opt = find_communities(n, iterations=6)
print(C)
print(q_opt)
pp.visualisation.plot(n, node_color=map_colors(n, C))

{'a': 1, 'b': 1, 'c': 1, 'd': 3, 'f': 5, 'g': 5, 'e': 3}
0.2160493827160494


### 2.3 Synthetic Network Generation

Create a simple *synthetic network* with a strong (and known) community structure as follows: 

1) Generate two networks $c1$ and $c2$ with $50$ nodes each and add $200$ links at random to each of the networks. For this, you can use the `numpy.random.choice` function introduced in the second tutorial.  
2) Use `pathpy`'s $+$-operator to combine the two networks to a single network with two connected components.   
3) Add $5$ links that randomly interconnect nodes across the two components $c1$ and $c2$, thus generating a connected network.

Visualise the network generated by this method.

In [ ]:
c_1 = pp.Network()
c_2 = pp.Network()

for i in range(50):
    c_1.add_node(str(i))
    c_2.add_node(str(50+i))

for i in range(200):
    c_1.add_edge(np.random.choice(list(c_1.nodes)), np.random.choice(list(c_1.nodes)))
    c_2.add_edge(np.random.choice(list(c_2.nodes)), np.random.choice(list(c_2.nodes)))
    
n_clustered = c_1 + c_2

for i in range(5):
    n_clustered.add_edge(np.random.choice(list(c_1.nodes)), np.random.choice(list(c_2.nodes)))
   
n_clustered

### 2.4 Heuristic optimisation vs. global optimum

Define a community partitioning that corresponds to the "ground truth" communities in the network from 2.3 and calculate the partition quality $Q$ for this optimal partitioning. Use your method from Task 2.2 to find the partitioning with maximal modularity. Compare this value to the ground truth and visualise the corresponding community structures.

In [ ]:
C_clustered = {}
for n in c_1.nodes:
    C_clustered[n] = 1
for n in c_2.nodes:
    C_clustered[n] = 2
    
q_opt = Q(n_clustered, C_clustered)
print('Optimal partition quality based on synthetic communities = {0}'.format(q_opt))

C, q_detected = find_communities(n_clustered, iterations=1500)
print('Optimal partition quality based on detected communities = {0}'.format(q_detected))

In [ ]:
pp.visualisation.plot(n_clustered, node_color=map_colors(n_clustered, C))

### 2.5 Community assortativity coefficient

Using the definition from lecture 2, slide 29, implement a function that computes the theoretical maximum modularity $Q_{max}$ fora given network and partition. Test your function for the toy example and try to reproduce the *community assortativity coefficient* reported on slide 29. Calculate the community assortativity coefficient for the synthetic example from Task 3.4 and compare it to the modularity of this network.

In [ ]:
def Qmax(network, C):
    m = network.ecount()
    qmax = 2*m
    for v in network.nodes:
        for w in network.nodes:
            if C[v] == C[w]:
                qmax -= network.nodes[v]['degree']*network.nodes[w]['degree']/(2*m)
    
    return qmax/(2*m)

In [ ]:
n = pp.Network()
n.add_edge('a', 'b')
n.add_edge('b', 'c')
n.add_edge('a', 'c')
n.add_edge('b', 'd')
n.add_edge('d', 'f')
n.add_edge('d', 'g')
n.add_edge('d', 'e')
n.add_edge('e', 'f')
n.add_edge('f', 'g')

communities = {}
for x in ['a', 'b', 'c']:
    communities[x] = 1
for x in ['d', 'e', 'f', 'g']:
    communities[x] = 2

q_max = Qmax(n, communities)
print(q_max)

C, q_opt = find_communities(n)
print(q_opt)

print("Community assortativity coefficient for toy example = ", q_opt/q_max)

In [ ]:
q_opt = Q(n_clustered, C_clustered)
q_max = Qmax(n_clustered, C_clustered)
print(q_max)
print(q_opt)
print("Community assortativity coefficient for synthetic network = ", q_opt/q_max)